# Train

In [1]:
import os
from box import Box
import torch.optim as optim

from utils import random_seed
from dataset import VAEData
from trainer import Trainer
from model import VAE

##############################ARGS##################################
args = {
    ######data#######
    'min_user_cnt' : 5,
    'min_movie_cnt' : 0,
    'n_heldout' : 3000,
    'target_prop' : 0.2,
    'min_movie_to_split' : 5,
    
    #####model######
    'hidden_dim' : 600,
    'latent_dim' : 200,
    'stnd_mixture_weight' : 3/20,
    'post_mixture_weight' : 3/4,
    'unif_mixture_weight' : 1/10,
    'dropout_ratio' : 0.5,
    
    #####optimizer####
    'lr' : 1e-4,
    'wd' : 0.00,
    
    ######trainer#####
    'batch_size' : 256,
    'epochs' : 50,
    'en_epochs' : 3,
    'de_epochs' : 1,
    'beta' : None,
    'gamma' : 0.005,
    'not_alter' : False,
    'ndcg_k' : 50,
    'recall_k' : 20,
    'verbose' : True,
    
    #####etc#######
    'base_dir' : '/opt/ml/input/data/',
    'random_seed' : 42,
    'device' : 'cuda'
}
args = Box(args)
##############################ARGS##################################

##############################PATHS##################################
dir_data = os.path.join(args.base_dir, 'train')
path_rating = os.path.join(dir_data, 'train_ratings.csv')
dir_output = os.path.join(os.getcwd(), 'output')
dir_preprocessing = os.path.join(os.getcwd(), 'data')

dir_file_path = {
    'dir_base': args.base_dir,
    'dir_data': dir_data,
    'rating': path_rating,
    'dir_output': dir_output,
    'dir_preprocessing' : dir_preprocessing
}
dir = Box(dir_file_path)
##############################PATHS##################################
    
random_seed(args.random_seed)

In [2]:
data_kwargs = {
    'data_dir' : dir.rating,
    'dir_preprocessing' : dir.dir_preprocessing,

    'min_user_cnt' : args.min_user_cnt,
    'min_movie_cnt' : args.min_movie_cnt,

    'n_heldout' : args.n_heldout,
    'target_prop' : args.target_prop,
    'min_movie_to_split' : args.min_movie_to_split
}

data = VAEData(**data_kwargs)

filter min...
user split...
getting data...
input target split...
input target split...
encoding...
encoding...
encoding...
encoding...
encoding...
encoding...
complete!


In [3]:
datasets = data.datasets # dict, {train_data, valid_data(input, target), test_data(input, target), inference_data}
input_dim = data.n_movies

model_kwargs = {
    'hidden_dim' : args.hidden_dim,
    'latent_dim' : args.latent_dim,
    'input_dim' : input_dim,

    'mixture_weights' : [args.stnd_mixture_weight,
                         args.post_mixture_weight,
                         args.unif_mixture_weight],
}

model = VAE(**model_kwargs).to(args.device)
model_best = VAE(**model_kwargs).to(args.device)

decoder_param = set(model.decoder.parameters())
encoder_param = set(model.encoder.parameters())

optimizer_encoder = optim.Adam(encoder_param, lr=args.lr, weight_decay=args.wd)
optimizer_decoder = optim.Adam(decoder_param, lr=args.lr, weight_decay=args.wd)

trainer_kwargs = {
    #model & optimizer
    'model' : model,
    'model_best' : model_best,
    'optimizer_encoder' : optimizer_encoder,
    'optimizer_decoder' : optimizer_decoder,

    #hyperparameters
    'batch_size' : args.batch_size,
    'epochs' : args.epochs,
    'en_epochs' : args.en_epochs,
    'de_epochs' : args.de_epochs,
    'beta' : args.beta,
    'gamma' : args.gamma,
    'dropout_ratio' : args.dropout_ratio,
    'not_alter' : args.not_alter,
    'ndcg_k' : args.ndcg_k,
    'recall_k' : args.recall_k,


    #datasets 
    'datasets' : datasets, 

    #etc
    'output_path' : dir.dir_output,
    'model_name' : 'RecVAE',
    'device' : args.device,
    'verbose' : args.verbose,

    # label encoder
    'user_encoder' : data.user_encoder,
    'item_encoder' : data.item_encoder
}

trainer = Trainer(**trainer_kwargs) 

In [ ]:
trainer.run()

[epoch 0/50 || valid_ndcg@50 : 0.1508 | best valid_ndcg : 0.1508 | train ndcg@50 : 0.3735
[epoch 1/50 || valid_ndcg@50 : 0.2736 | best valid_ndcg : 0.2736 | train ndcg@50 : 0.5443
[epoch 2/50 || valid_ndcg@50 : 0.3085 | best valid_ndcg : 0.3085 | train ndcg@50 : 0.6055
[epoch 3/50 || valid_ndcg@50 : 0.3328 | best valid_ndcg : 0.3328 | train ndcg@50 : 0.6401
[epoch 4/50 || valid_ndcg@50 : 0.3460 | best valid_ndcg : 0.3460 | train ndcg@50 : 0.6579
[epoch 5/50 || valid_ndcg@50 : 0.3551 | best valid_ndcg : 0.3551 | train ndcg@50 : 0.6699
[epoch 6/50 || valid_ndcg@50 : 0.3599 | best valid_ndcg : 0.3599 | train ndcg@50 : 0.6794
[epoch 7/50 || valid_ndcg@50 : 0.3642 | best valid_ndcg : 0.3642 | train ndcg@50 : 0.6888
[epoch 8/50 || valid_ndcg@50 : 0.3681 | best valid_ndcg : 0.3681 | train ndcg@50 : 0.6922
[epoch 9/50 || valid_ndcg@50 : 0.3713 | best valid_ndcg : 0.3713 | train ndcg@50 : 0.6965
[epoch 10/50 || valid_ndcg@50 : 0.3746 | best valid_ndcg : 0.3746 | train ndcg@50 : 0.7014
[epoch 11

# Inference

In [ ]:
def inference(trainer, data, k, metric):
    
    if metric == 'recall':
        model = trainer.recall_best_model
    if metric == 'ndcg':
        model = trainer.ndcg_best_model
        
    input_data = trainer.inference_data
    users = range(data.n_users)

    model.eval()
    with torch.no_grad():

        input_data = trainer._sparse2Tensor(input_data).to(trainer.device)
        prediction, _, _ = model(input_data)
        prediction[torch.nonzero(input_data, as_tuple=True)] = -np.inf
        scores, movies = torch.topk(prediction, dim=1, k=k)
        
        users = np.tile(users, (k,1)).T
        user_list = np.concatenate([user for user in users])
        score_list = torch.cat([score for score in scores])
        movie_list = torch.cat([movie for movie in movies])
    
    user_decoder = {value : key for (key, value) in trainer.user_encoder.items()}
    item_decoder = {value : key for (key, value) in trainer.item_encoder.items()}
    
    inference_df = pd.DataFrame()
    inference_df['user'] = user_list
    inference_df['item'] = movie_list.cpu().numpy()
    inference_df['score'] = score_list.cpu().numpy()
    
    inference_df['user'] = inference_df['user'].apply(lambda x : user_decoder[x])
    inference_df['item'] = inference_df['item'].apply(lambda x : item_decoder[x])

    return inference_df

In [ ]:
inference_df = inference(trainer, data, k=10, metric='ndcg')
inference_df = inference_df.sort_values(['user', 'score'], ascending=[True, False])